In [24]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### IOU

In [25]:
# def calculate_iou(true_mask, predicted_mask):
#     #convert true mask to gray scale
#     true_mask_gray=cv2.cvtColor(true_mask,cv2.COLOR_BGR2GRAY)
#     #convert predicted mask to gray scale
#     predicted_mask_gray=cv2.cvtColor(predicted_mask,cv2.COLOR_BGR2GRAY)
#     #find contours for true mask
#     true_contours,_=cv2.findContours(true_mask_gray,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#     #find contours for predicted mask
#     predicted_contours,_=cv2.findContours(predicted_mask_gray,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#     #initialize iou
#     iou=0
#     #calculate iou only if both true and predicted contours are present
#     if len(true_contours)>0 and len(predicted_contours)>0:
#         #get intersection contour area
#         intersection_area=cv2.contourArea(cv2.convexHull(cv2.bitwise_and(true_mask_gray,predicted_mask_gray)))

#         #get union contour area
#         union_area=cv2.contourArea(cv2.convexHull(cv2.bitwise_or(true_mask_gray,predicted_mask_gray)))

#         #calculate iou
#         iou=intersection_area/union_area

#     else:
#         iou=0

#     return iou

In [26]:
def calculate_iou(true_mask, predicted_mask):
    # Convert true mask to gray scale
    true_mask_gray = cv2.cvtColor(true_mask, cv2.COLOR_BGR2GRAY)
    # Convert predicted mask to gray scale
    predicted_mask_gray = cv2.cvtColor(predicted_mask, cv2.COLOR_BGR2GRAY)
    
    # Get intersection image
    intersection = cv2.bitwise_and(true_mask_gray, predicted_mask_gray)
    # Get union image
    union = cv2.bitwise_or(true_mask_gray, predicted_mask_gray)
    
    # Find contours in the intersection and union images
    contours_intersection, _ = cv2.findContours(intersection, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    contours_union, _ = cv2.findContours(union, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize IoU
    iou = 0
    
    # Calculate IoU only if both true and predicted contours are present
    if len(contours_intersection) > 0 and len(contours_union) > 0:
        # Get intersection contour area
        intersection_area = cv2.contourArea(cv2.convexHull(np.concatenate(contours_intersection)))
        # Get union contour area
        union_area = cv2.contourArea(cv2.convexHull(np.concatenate(contours_union)))
        
        # Calculate IoU
        iou = intersection_area / union_area
    else:
        iou = 0

    return iou

### Dice

In [27]:
# #calculate dice score
# def calculate_dice_score(true_mask, predicted_mask):
#     #convert true mask to gray scale
#     true_mask_gray=cv2.cvtColor(true_mask,cv2.COLOR_BGR2GRAY)
#     #convert predicted mask to gray scale
#     predicted_mask_gray=cv2.cvtColor(predicted_mask,cv2.COLOR_BGR2GRAY)
#     #find contours for true mask
#     true_contours,_=cv2.findContours(true_mask_gray,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#     #find contours for predicted mask
#     predicted_contours,_=cv2.findContours(predicted_mask_gray,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#     dice_score=0
#     #calculate dice score only if both true and predicted contours are present
#     if len(true_contours)>0 and len(predicted_contours)>0:
#         #calculate dice score
#         #intersection area
#         intersection_area=cv2.contourArea(cv2.convexHull(cv2.bitwise_and(true_mask_gray,predicted_mask_gray)))
#         #area of true mask
#         true_area=cv2.contourArea(cv2.convexHull(true_mask_gray))
#         #area of predicted mask
#         predicted_area=cv2.contourArea(cv2.convexHull(predicted_mask_gray))
#         #dice score
#         dice_score=2*intersection_area/(true_area+predicted_area)

#     else:
#         dice_score=0

#     return dice_score

In [28]:
def calculate_dice_score(true_mask, predicted_mask):
    # Convert true mask to gray scale
    true_mask_gray = cv2.cvtColor(true_mask, cv2.COLOR_BGR2GRAY)
    # Convert predicted mask to gray scale
    predicted_mask_gray = cv2.cvtColor(predicted_mask, cv2.COLOR_BGR2GRAY)
    
    # Get intersection image
    intersection = cv2.bitwise_and(true_mask_gray, predicted_mask_gray)
    
    # Find contours in the intersection and individual masks
    contours_intersection, _ = cv2.findContours(intersection, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours_intersection) == 0:
        return 0
    contours_true, _ = cv2.findContours(true_mask_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_pred, _ = cv2.findContours(predicted_mask_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    dice_score = 0
    
    # Calculate Dice score only if both true and predicted contours are present
    if len(contours_true) > 0 and len(contours_pred) > 0:
        # Calculate intersection area
        intersection_area = cv2.contourArea(cv2.convexHull(np.concatenate(contours_intersection)))
        # Calculate area of true mask
        true_area = cv2.contourArea(cv2.convexHull(np.concatenate(contours_true)))
        # Calculate area of predicted mask
        predicted_area = cv2.contourArea(cv2.convexHull(np.concatenate(contours_pred)))
        
        # Calculate Dice score
        dice_score = 2 * intersection_area / (true_area + predicted_area)
    else:
        dice_score = 0

    return dice_score

### pixel wise accuracy

In [29]:
#pixel wise accuracy

def calculate_pixel_wise_accuracy(true_mask, predicted_mask):
    
    #convert true mask to gray scale
    true_mask_gray=cv2.cvtColor(true_mask,cv2.COLOR_BGR2GRAY)
    #convert predicted mask to gray scale
    predicted_mask_gray=cv2.cvtColor(predicted_mask,cv2.COLOR_BGR2GRAY)
    #calculate pixel wise accuracy
    #number of pixels in true mask
    true_pixels=np.sum(true_mask_gray==255)
    #number of pixels in predicted mask
    predicted_pixels=np.sum(predicted_mask_gray==255)
    #number of pixels in true mask and predicted mask
    true_predicted_pixels=np.sum((true_mask_gray==255) & (predicted_mask_gray==255))
    #pixel wise accuracy
    pixel_wise_accuracy=true_predicted_pixels/(true_pixels+predicted_pixels-true_predicted_pixels)

    return pixel_wise_accuracy

### Bbox

In [30]:
#
benign_predictions_dir=os.path.join('..', 'BreastSAM impl', 'SAM_predictions_BBox', 'benign')
benign_gt_dir=os.path.join('..','Dataset_BUSI_with_GT','benign')

#file names in benign_gt_dir =benign (1)_mask.png
#file names in benign_predictions_dir =benign (1)_mask_prediction.png


list_of_benign_predictions_dir=os.listdir(benign_predictions_dir)

df=pd.DataFrame(columns=['File','IOU','Dice Score','Pixel Wise Accuracy'])

for files in list_of_benign_predictions_dir:
    
    #read true mask
    true_mask=cv2.imread(os.path.join(benign_gt_dir,files.replace('_mask_prediction.png','_mask.png')))
    #read predicted mask
    predicted_mask=cv2.imread(os.path.join(benign_predictions_dir,files))
    
    #calculate iou
    iou=calculate_iou(true_mask,predicted_mask)
    #calculate dice score
    dice_score=calculate_dice_score(true_mask,predicted_mask)
    #calculate pixel wise accuracy
    pixel_wise_accuracy=calculate_pixel_wise_accuracy(true_mask,predicted_mask)

    #dont use append method to add rows to dataframe

    df.loc[len(df)]=[files,iou,dice_score,pixel_wise_accuracy]
    


print(df)

#save the dataframe to a csv file
df.to_csv(os.path.join('..','BreastSAM impl','benign_metrics.csv'),index=False)

                                 File       IOU  Dice Score  \
0      benign (1)_mask_prediction.png  0.838826    0.908780   
1     benign (10)_mask_prediction.png  0.945461    0.971808   
2    benign (100)_mask_prediction.png  0.826918    0.905807   
3    benign (101)_mask_prediction.png  0.874150    0.932728   
4    benign (102)_mask_prediction.png  0.929005    0.963614   
..                                ...       ...         ...   
431   benign (94)_mask_prediction.png  0.799413    0.886248   
432   benign (95)_mask_prediction.png  0.849863    0.918625   
433   benign (96)_mask_prediction.png  0.842329    0.914207   
434   benign (97)_mask_prediction.png  0.858714    0.924011   
435   benign (98)_mask_prediction.png  0.794897    0.884030   

     Pixel Wise Accuracy  
0               0.839858  
1               0.939535  
2               0.827932  
3               0.863964  
4               0.921084  
..                   ...  
431             0.805434  
432             0.801372  


In [31]:
def calculate_metrics(benign_gt_dir, benign_predictions_dir,csv_file_name):
    list_of_benign_predictions_dir = os.listdir(benign_predictions_dir)

    df = pd.DataFrame(columns=['File', 'IOU', 'Dice Score', 'Pixel Wise Accuracy'])

    for files in list_of_benign_predictions_dir:
        # read true mask
        #print(files)
        true_mask = cv2.imread(os.path.join(benign_gt_dir, files.replace('_mask_prediction.png', '_mask.png')))
        # read predicted mask
        predicted_mask = cv2.imread(os.path.join(benign_predictions_dir, files))

        # calculate iou
        iou = calculate_iou(true_mask, predicted_mask)
        # calculate dice score
        dice_score = calculate_dice_score(true_mask, predicted_mask)
        # calculate pixel wise accuracy
        pixel_wise_accuracy = calculate_pixel_wise_accuracy(true_mask, predicted_mask)

        df.loc[len(df)] = [files, iou, dice_score, pixel_wise_accuracy]

    print(df.head())

    # save the dataframe to a csv file
    df.to_csv(csv_file_name, index=False)
    
    
    average_dice_score=np.mean(df['Dice Score'])
    average_iou=np.mean(df['IOU'])
    average_pixel_wise_accuracy=np.mean(df['Pixel Wise Accuracy'])
    
    return average_dice_score,average_iou,average_pixel_wise_accuracy

## only box prompt

In [32]:
benign_predictions_dir=os.path.join('..', 'BreastSAM impl', 'SAM_predictions_BBox', 'benign')
benign_gt_dir=os.path.join('..','Dataset_BUSI_with_GT','benign')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir,benign_predictions_dir,os.path.join('..','BreastSAM impl','benign_box_metrics.csv'))

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

                               File       IOU  Dice Score  Pixel Wise Accuracy
0    benign (1)_mask_prediction.png  0.838826    0.908780             0.839858
1   benign (10)_mask_prediction.png  0.945461    0.971808             0.939535
2  benign (100)_mask_prediction.png  0.826918    0.905807             0.827932
3  benign (101)_mask_prediction.png  0.874150    0.932728             0.863964
4  benign (102)_mask_prediction.png  0.929005    0.963614             0.921084
Average Dice Score: 0.8966026793007775
Average IOU: 0.8219298047116779
Average Pixel Wise Accuracy: 0.7751496027403703


In [33]:
malignant_predictions_dir=os.path.join('..', 'BreastSAM impl', 'SAM_predictions_BBox', 'malignant')
malignant_gt_dir=os.path.join('..','Dataset_BUSI_with_GT','malignant')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir,malignant_predictions_dir,os.path.join('..','BreastSAM impl','malignant_box_metrics.csv'))

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

                                  File       IOU  Dice Score  \
0    malignant (1)_mask_prediction.png  0.848478    0.910377   
1   malignant (10)_mask_prediction.png  0.728952    0.841666   
2  malignant (100)_mask_prediction.png  0.726468    0.837183   
3  malignant (101)_mask_prediction.png  0.540087    0.700621   
4  malignant (102)_mask_prediction.png  0.572006    0.727526   

   Pixel Wise Accuracy  
0             0.768956  
1             0.690886  
2             0.693924  
3             0.633904  
4             0.579419  
Average Dice Score: 0.8480665238266606
Average IOU: 0.7496548193711752
Average Pixel Wise Accuracy: 0.7083671832802122


## with box and point prompt

In [34]:
benign_gt_dir=os.path.join('..','Dataset_BUSI_with_GT','benign')
benign_point_box_dir=os.path.join('..','BreastSAM impl','SAM_predictions_BBoxPoint','benign')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir,benign_point_box_dir,'benign_BBoxPoint_metrics.csv')

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

                               File       IOU  Dice Score  Pixel Wise Accuracy
0    benign (1)_mask_prediction.png  0.836117    0.905845             0.838825
1   benign (10)_mask_prediction.png  0.946889    0.972232             0.941567
2  benign (100)_mask_prediction.png  0.825503    0.904992             0.826915
3  benign (101)_mask_prediction.png  0.876017    0.933730             0.866155
4  benign (102)_mask_prediction.png  0.931304    0.964796             0.925291
Average Dice Score: 0.897176626447334
Average IOU: 0.8223667122607278
Average Pixel Wise Accuracy: 0.79010434918551


In [35]:
malignant_gt_dir=os.path.join('..','Dataset_BUSI_with_GT','malignant')
malignant_point_box_dir=os.path.join('..','BreastSAM impl','SAM_predictions_BBoxPoint','malignant')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir,malignant_point_box_dir,'malignant_BBoxPoint_metrics.csv')

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)


                                  File       IOU  Dice Score  \
0    malignant (1)_mask_prediction.png  0.836582    0.896123   
1   malignant (10)_mask_prediction.png  0.674077    0.792074   
2  malignant (100)_mask_prediction.png  0.685972    0.813638   
3  malignant (101)_mask_prediction.png  0.538556    0.699385   
4  malignant (102)_mask_prediction.png  0.596210    0.746671   

   Pixel Wise Accuracy  
0             0.754960  
1             0.664978  
2             0.616424  
3             0.630228  
4             0.576915  
Average Dice Score: 0.8443522558945744
Average IOU: 0.7429058788803667
Average Pixel Wise Accuracy: 0.7155563276327351


### Point 

In [36]:
benign_gt_dir=os.path.join('..','Dataset_BUSI_with_GT','benign')
benign_point_box_dir=os.path.join('..','BreastSAM impl','SAM_predictions_Point','benign')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir,benign_point_box_dir,'benign_Point_metrics.csv')

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

                               File       IOU  Dice Score  Pixel Wise Accuracy
0    benign (1)_mask_prediction.png  0.840455    0.911286             0.846985
1   benign (10)_mask_prediction.png  0.947690    0.972580             0.943227
2  benign (100)_mask_prediction.png  0.820798    0.901956             0.822899
3  benign (101)_mask_prediction.png  0.883074    0.937726             0.872422
4  benign (102)_mask_prediction.png  0.630506    0.767371             0.920207
Average Dice Score: 0.6939704876595888
Average IOU: 0.6100689249457486
Average Pixel Wise Accuracy: 0.5896810091898083


In [37]:
malignant_gt_dir=os.path.join('..','Dataset_BUSI_with_GT','malignant')
malignant_point_box_dir=os.path.join('..','BreastSAM impl','SAM_predictions_Point','malignant')

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir,malignant_point_box_dir,'malignant_Point_metrics.csv')

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)


                                  File       IOU  Dice Score  \
0    malignant (1)_mask_prediction.png  0.657899    0.793695   
1   malignant (10)_mask_prediction.png  0.084546    0.155910   
2  malignant (100)_mask_prediction.png  0.306216    0.468812   
3  malignant (101)_mask_prediction.png  0.499169    0.665529   
4  malignant (102)_mask_prediction.png  0.510866    0.676209   

   Pixel Wise Accuracy  
0             0.675349  
1             0.088316  
2             0.355373  
3             0.604435  
4             0.513201  
Average Dice Score: 0.5763191115128532
Average IOU: 0.44829930329605333
Average Pixel Wise Accuracy: 0.46138997781788166
